# Connecting To PostGres with Python

In this notebook, I want to showcase how the psycopg2 library can help us execute SQL-queries directly in a python terminal, IDE or notebook. 

The 'SuperMarketTransactions' csv-file has been uploaded to my localhost on PostGres. It includes data over fictional transactions in a pan-american sueprmarket, and we will use SQL to retreive insights on the behaviour of these customers


In [30]:
import psycopg2
import pandas as pd

Every PostGres database has the following specs that are needed to connect to it

In [12]:
DB_NAME = "TestData"
DB_USER = "postgres"
DB_HOST = "localhost"
DB_PORT = "5432"

psycopg2 creates a "cursor" with which we can "fetch" data from the DB

In [197]:
try:
    conn = psycopg2.connect(database = DB_NAME, user = DB_USER,
                            host = DB_HOST, port = DB_PORT)
    print('connected')
except:
    print('not connected')

connected


Now that we've established a connection, lets write a function to retreive the data based an a Query (here called Q). Then, we fetch the raw SQL and turn it into a dataframe with it's accompanying column names  

In [201]:
def SQL_Query(Q):
    
    try:
        conn = psycopg2.connect(database = DB_NAME, user = DB_USER,
                            host = DB_HOST, port = DB_PORT)
        print('connected')
    except:
        print('not connected')
    
    cur = conn.cursor()
    cur.execute(Q)
    rows = cur.fetchall()
    Data = pd.DataFrame(rows)
    colname= [desc[0] for desc in cur.description]
    Data.columns = colname
    return(Data)
    curr.close()
    conn.close()

Let's test it out! We start simply and call the average number of children of the customers by country the operate in

In [202]:
QUERY = '''SELECT country,round(avg(children),2) as avg_children
           FROM sales.super 
           GROUP BY country 
           LIMIT 5 '''

In [203]:
SQL_Query(QUERY)

connected


,country,avg_children
0,Canada,2.51
1,Mexico,2.54
2,USA,2.57


It works! We see that even the alias was included. Let's try something more complicated, like incorporating a window function. In the Query below, we look at total revenue per city, ranked from highest to lowest, but also the ranking within country (Hidalgo bering the top performer in Mexico but only 7th overall.

In [204]:
QUERY1 = '''SELECT country, city, round(sum(revenue),2) as tot_rev,
                   RANK() OVER(ORDER BY round(sum(revenue),2)DESC),
                   RANK() OVER(PARTITION BY Country ORDER BY round(sum(revenue),2) DESC)
            FROM sales.super
            GROUP BY city, Country
            ORDER BY 3 DESC
            LIMIT 10'''

In [205]:
SQL_Query(QUERY1)

connected


,country,city,tot_rev,rank,rank
0,USA,Salem,4647.42,1,1
1,USA,Tacoma,4428.90,2,2
2,USA,Portland,3118.70,3,3
3,USA,Bremerton,3060.50,4,4
4,USA,Seattle,2972.34,5,5
5,USA,Los Angeles,2727.30,6,6
6,Mexico,Hidalgo,2427.70,7,1
7,USA,San Diego,2275.54,8,7
8,Mexico,Merida,2239.84,9,2
9,USA,Spokane,2193.92,10,8


What drives profitability in these stores? We can have a look at the customerbase to get a better understanding


In [218]:
Query2 = '''SELECT state_or_province,
                   round(avg(numerical_income),2) as avg_income,
                   round(avg(children),2) as avg_children,
                   round(avg(units_sold),2) as avg_basket_size,
                   round(avg(revenue),2) as avg_spend,
                   round((avg(CASE WHEN homeowner ='Y' THEN 1
                        ELSE 0 END) * 100),2) as home_ownership_percentage
            FROM sales.super
            GROUP BY state_or_province
            ORDER BY avg_spend DESC
            LIMIT 10''' 

In [219]:
SQL_Query(Query2)

connected


,state_or_province,avg_income,avg_children,avg_basket_size,avg_spend,home_ownership_percentage
0,Guerrero,52765.96,2.74,4.38,15.97,55.32
1,DF,55268.82,2.51,4.16,15.06,62.37
2,Yucatan,62400.00,2.59,4.29,14.93,68.00
3,OR,57777.78,2.52,4.11,14.88,60.92
4,BC,57631.58,2.51,3.88,14.83,64.47
5,Zacatecas,57517.24,2.41,3.97,14.54,63.45
6,WA,59388.49,2.59,3.99,12.85,63.49
7,CA,59860.63,2.57,4.16,12.80,66.20
8,Veracruz,69130.43,2.63,3.93,12.73,65.22
9,Jalisco,50000.00,2.79,2.57,7.88,35.71
